In [ ]:
import torch
import numpy as np

def le(x, y, t):
    d = torch.norm(x - y, dim=2)
    d = d / d[:, :, 0:1]  
    d = torch.log(d) 
    d = d / t.unsqueeze(0).unsqueeze(1)
    return d

def MLE(traj, t, eps=0.1):
    x_amount, y_amount, x_dim, t_dim = traj.shape
    traj_init = traj[:, :, :, 0:1]  # select initial values
    traj_norm = torch.norm(traj_init.unsqueeze(2).unsqueeze(3) - traj_init.unsqueeze(0).unsqueeze(1), dim=2)
    mask = (traj_norm < eps) & (traj_norm != 0)  

    le_vals = le(traj.unsqueeze(2).unsqueeze(3), traj.unsqueeze(0).unsqueeze(1), t)
    le_vals_masked = torch.where(mask.unsqueeze(-1), le_vals, torch.tensor(float('nan')))
    le_avg = torch.nanmean(le_vals_masked, dim=(2, 3))
    return le_avg


def evolution_2d(x, t, l=1.):
    norm_x = torch.linalg.norm(x, dim = -1, keepdim=True)  # shape (x_amount,x_amount,1)
    indices_below = (norm_x < 1).expand_as(x)  # shape (x_amount,x_amount,dim_x)

    t = t[None, None, None, :]  # shape (1,1,1,3)
    x_extended = x.unsqueeze(-1).expand(-1, -1, -1, len(t))  # shape (x_amount,x_amount,dim_x,len_t)

    result_below = torch.exp(-l * t) * x_extended  # shape (x_amount,x_amount,dim_x,len_t)
    result_above = torch.exp(l * t) * x_extended  # shape (x_amount,x_amount,dim_x,len_t)

    # Use the boolean mask to select values from result_below or result_above
    result = torch.where(indices_below.unsqueeze(-1), result_below, result_above)  # shape (x_amount,x_amount,dim_x,len_t)

    return result


x_amount = 30
x = torch.linspace(-2,2,x_amount)
y = torch.linspace(-2,2,x_amount)
X, Y = torch.meshgrid(x, y)
inputs = torch.stack([X,Y], dim=-1)

t = torch.linspace(0,1,2)
trajectories = evolution_2d(inputs, t)
t_indices = torch.tensor([0,-1])

trajectories = trajectories[t_indices] 
trajectories = trajectories.permute(1,2,3,0)

eps = 0.6
output = MLE(trajectories,t,eps)
print(output)